# Job planet 채용 공고 scraping 스크립트

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

import pandas as pd

import time
from tqdm.notebook import tqdm

In [10]:
baseUrl = 'https://www.jobplanet.co.kr'
scroll_num = 300 # 9 + 9 * scroll_num of hrefs in maximum
major_classification = '개발'
minor_classification = ['개발 전체']

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

In [3]:
# Function for scrolling

def scroll(driver, scroll_limit):
    '''
    Each scroll gets 9 additional references and it takes 1 second of timeout.
    '''
    n = 1
    prev_height = driver.execute_script('return document.body.scrollHeight')
    while n <= scroll_limit:
        print('scrolling', n, '/', scroll_limit)
        n += 1
        driver.execute_script('window.scrollBy(0, 3000)')
        time.sleep(1)
        curr_height = driver.execute_script('return document.body.scrollHeight')
        if curr_height == prev_height:
            time.sleep(5)
            curr_height = driver.execute_script('return document.body.scrollHeight')
            if curr_height == prev_height:
                break
        prev_height = curr_height
    time.sleep(5)

In [11]:
# Making a list of target hrefs (job_descriptions)

with webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) as driver:
  driver.implicitly_wait(5)
  driver.get(baseUrl + '/job')

  WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'jply_checkbox_item')))
  driver.find_element(By.XPATH, "//*[text()='직종']").click()
  driver.find_element(By.XPATH, f"//*[text()='{major_classification}']").click()
  for minor in minor_classification:
    driver.find_element(By.XPATH, f"//*[text()='{minor}']").click()
  driver.find_element(By.XPATH, "//*[text()='적용']").click()

  time.sleep(5)

  scroll(driver, scroll_num) # get (1 + 3) * 9 = 36 job descriptions

  print('collecting hrefs')
  job_descriptions = [a.get_attribute('href') for a in driver.find_elements(By.TAG_NAME, 'a') if a.get_attribute('href') and baseUrl + '/job/search' in a.get_attribute('href')]

print(len(job_descriptions), job_descriptions)

scrolling 1 / 300
scrolling 2 / 300
scrolling 3 / 300
scrolling 4 / 300
scrolling 5 / 300
scrolling 6 / 300
scrolling 7 / 300
scrolling 8 / 300
scrolling 9 / 300
scrolling 10 / 300
scrolling 11 / 300
scrolling 12 / 300
scrolling 13 / 300
scrolling 14 / 300
scrolling 15 / 300
scrolling 16 / 300
scrolling 17 / 300
scrolling 18 / 300
scrolling 19 / 300
scrolling 20 / 300
scrolling 21 / 300
scrolling 22 / 300
scrolling 23 / 300
scrolling 24 / 300
scrolling 25 / 300
scrolling 26 / 300
scrolling 27 / 300
scrolling 28 / 300
scrolling 29 / 300
scrolling 30 / 300
scrolling 31 / 300
scrolling 32 / 300
scrolling 33 / 300
scrolling 34 / 300
scrolling 35 / 300
scrolling 36 / 300
scrolling 37 / 300
scrolling 38 / 300
scrolling 39 / 300
scrolling 40 / 300
scrolling 41 / 300
scrolling 42 / 300
scrolling 43 / 300
scrolling 44 / 300
scrolling 45 / 300
scrolling 46 / 300
scrolling 47 / 300
scrolling 48 / 300
scrolling 49 / 300
scrolling 50 / 300
scrolling 51 / 300
scrolling 52 / 300
scrolling 53 / 300
sc

In [5]:
def find_by_text(html, text, data_obj, col_name, parent=None, h3=None):
    finding = html.find(name=h3, text=text)

    if finding and parent:
        finding = finding.find_parent('dt')

    if finding:
        data_obj[col_name].append(finding.find_next_sibling().text)
    else:
        data_obj[col_name].append('')

In [15]:
# Scrapping recruitment data from all hrefs collected

extracted_data = {
    'href': [],
    'title': [],
    'company_name': [],
    'job_location': [],
    'job_classification': [],
    'due_date': [],
    'work_experience': [],
    'skills': [],
    'company_intro': [],
    'main_task': [],
    'qualification': [],
    'preference': [],
    'procedure': [],
    'benefits': [],
    'job_location_detail': [],
    'detail_description': [],
}


with webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) as driver:
    driver.implicitly_wait(5)
    for i in tqdm(range(len(job_descriptions))):
      driver.get(job_descriptions[i])
      WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'job_apply_section')))

      soup = BeautifulSoup(driver.page_source, 'html.parser')
      apply = soup.find('div', 'job_apply_section')
      detail = soup.find('section', 'recruitment-detail')
      summary = detail.find('dl', 'recruitment-summary__dl')

      extracted_data['href'].append(job_descriptions[i])
      extracted_data['title'].append(apply.find('h1', 'ttl').text)
      extracted_data['company_name'].append(apply.find('span', 'company_name').text)
      extracted_data['job_location'].append(apply.find('span', 'job_location').text)
      find_by_text(summary, '직무', extracted_data, 'job_classification', parent=True)
      find_by_text(summary, '마감일', extracted_data, 'due_date', parent=True)
      find_by_text(summary, '경력', extracted_data, 'work_experience', parent=True)
      find_by_text(summary, '스킬', extracted_data, 'skills', parent=True)
      find_by_text(detail, '기업 소개', extracted_data, 'company_intro', h3=True)
      find_by_text(detail, '주요 업무', extracted_data, 'main_task', h3=True)
      find_by_text(detail, '자격 요건', extracted_data, 'qualification', h3=True)
      find_by_text(detail, '우대사항', extracted_data, 'preference', h3=True)
      find_by_text(detail, '채용 절차', extracted_data, 'procedure', h3=True)
      find_by_text(detail, '복리후생', extracted_data, 'benefits', h3=True)
      find_by_text(detail, '회사위치', extracted_data, 'job_location_detail', h3=True)
      extracted_data['detail_description'].append(detail)


  0%|          | 0/1242 [00:00<?, ?it/s]

C:\Users\TimJimTangtong\AppData\Local\Temp\ipykernel_26208\808487747.py:2: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  finding = html.find(name=h3, text=text)


In [16]:
extracted_df = pd.DataFrame(extracted_data)
extracted_df.head()

,href,title,company_name,job_location,job_classification,due_date,work_experience,skills,company_intro,main_task,qualification,preference,procedure,benefits,job_location_detail,detail_description
0,https://www.jobplanet.co.kr/job/search?posting...,B2B 백엔드 엔지니어,(주)마이프랜차이즈,서울,백엔드 개발,2023.07.24 D-1,경력무관,"백엔드, B2B, MongoDB, node js",＜마이프랜차이즈가 오프라인 창업 시장을 바꿉니다.＞\n\n코로나19 이후 어려운 환...,• 마이프차 Partner 대상 서비스 백엔드 개선 및 새 기능 개발\n• 각 상...,• 3년 이상의 상용 서비스 백엔드 개발 경험 또는 이에 준하는 역량 보유자\n• ...,• 프로젝트 세팅부터 서비스 런칭까지 전 과정에 대한 경험이 있는 분\n• MSA ...,＜채용은 이렇게 진행됩니다＞\n1. 서류 전형\n : 마이프랜차이즈는 정해진 양식 ...,"• 시차출퇴근제 (08~10시 자율 출근, 일 8시간 근무)\n• 수습 기간 종료 ...","서울시 강남구 강남대로92길 19, 여원빌딩 11층","[[[요약], [<dt class=""recruitment-summary__dt""><..."
1,https://www.jobplanet.co.kr/job/search?posting...,QA Engineer (팀리드),잡플래닛,서울,QA(Quality Assurance),2023.08.21 D-29 (채용시 마감),7년 이상,"java, QA, c, C++, python, istqb, 테스트, 검증, JIRA...",더 나은 선택은 없을까?\n 지금 이 순간에도 잡플래닛을 방문하는 월 300만명의 ...,"잡플래닛의 QA팀은 팀내에서 지식 공유가 이뤄지되, 업무는 비즈니스별 스쿼드로 배치...",- 직무 경력 7년 이상이신 분\n - 팀장/리드급으로 근무하며 조직을 관리한 경...,- 조직의 역량강화를 위한 지식전파 경험이 있는 분\n - ISTQB 등 테스트 ...,전형 프로세스\n - 서류전형 -> 1차 실무 인터뷰 -> 2차 협업부서 인터뷰 ...,- 분 단위 시차 출퇴근제 (오전 7시~ 오전 10시 사이 자율 출퇴근)\n - ...,"서울 강남구 테헤란로 415, 307호","[[[요약], [<dt class=""recruitment-summary__dt""><..."
2,https://www.jobplanet.co.kr/job/search?posting...,QA 엔지니어 (3년 이상),(주)오픈서베이,서울,QA(Quality Assurance),2023.08.11 D-19 (채용시 마감),3년 이상,"QA, 테스트, SW QA, JIRA",【 오픈서베이와 팀을 소개합니다! 】\n소비자 데이터 플랫폼 오픈서베이는 기술로 설...,오픈서베이의 QA팀은 테스트뿐 아니라 출시되는 제품의 최종 품질을 보증하며 개발 프...,【 이런 역량을 가진 분을 찾습니다 】\n• 3년 이상의 웹 서비스 또는 모바일 앱...,【 이런 역량을 가진 분이면 더 좋습니다 】\n• 서비스의 Kick-off부터 Re...,【 오픈서베이 합류 과정은 다음과 같습니다 】\n• 서류지원 ＞ 1차 인터뷰 ＞ 2...,1. 집중할 수 있는 업무 환경을 제공합니다\n• 쾌적한 오피스 : 강남역 도보 3...,서울특별시 강남구 강남대로84길 13 KR타워,"[[[요약], [<dt class=""recruitment-summary__dt""><..."
3,https://www.jobplanet.co.kr/job/search?posting...,Back-end 개발자(신규 서비스 개발 및 운영),(주)트레드링스,서울,웹개발,2023.08.21 D-29,5년 이상,"java, Spring, SQL, java spring 백엔드",지금 트레드링스에 합류하시면 채용축하금을 드립니다!\n자세한 내용은 트레드링스 채용...,[Back-end 개발자(신규 서비스 개발 및 운영)는 이런 일을 해요]\n• 수출...,"[이런 분과 함께 하고 싶어요]\n• Java, Spring Boot 경력 5년 이...","[이런 경험이 있으면 더 좋아요]\n• 클라우드 기반의 서비스, k8s, VM, 리...",,[몰입을 위한 모든 것을 지원하는 트레드링스의 혜택을 소개해요]\n• 수평적 조직문...,"서울특별시 서초구 서초대로77길 17, 13층","[[[요약], [<dt class=""recruitment-summary__dt""><..."
4,https://www.jobplanet.co.kr/job/search?posting...,웹 플랫폼서비스 백엔드 Python 서버 개발자,(주)히츠,서울,백엔드 개발,2023.08.10 D-18,1년 이상,"python, Linux, Kubernetes, Golang",- 함께할 팀을 소개합니다.\n\n히츠 플랫폼 개발팀은 새로운 기술을 두려워하지 않...,[이런 업무를 하시게 돼요]\n\n1. Cloud-native 접근 방식으로 서비스...,1. (필수) 1년 이상 백앤드 개발경력\n\n2. Python을 이용한 서비스 개...,• Golang을 이용한 서비스 개발 경험\n• Linux 환경에서의 서비스 개발 ...,,"＜구성원의 업무 능률 향상을 위해 ＞\n\n- 고사양 업무 장비(노트북, 모니터 등...","서울 강남구 테헤란로 124, 902호","[[[요약], [<dt class=""recruitment-summary__dt""><..."


In [17]:
extracted_df.to_csv('./extracted.csv', index=False)